# FaceMask Detector

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import cv2

In [2]:
img_size = 224
threshold = 0.5
frame_dimension = 600
red_color = (0, 0, 255)
green_color = (0, 255, 0)
mask_label = "Mask"
no_mask_label = "No Mask"
quit_key = "q"

In [3]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2] #take dimensions of the frame
    blob = cv2.dnn.blobFromImage(frame, 1.0, (img_size, img_size), (104.0, 177.0, 123.0))#construct a blob

    faceNet.setInput(blob) 
    detections = faceNet.forward()#detect faces
    
    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):        
        confidence = detections[0, 0, i, 2]
        
        if confidence > threshold:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h]) #x and y coordinates of the box
            (startX, startY, endX, endY) = box.astype("int")

            # make sure that the bounding boxes do not fall outside the dimension of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (img_size, img_size))
            face = img_to_array(face)            
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    
    if len(faces) > 0: #make predictions only if faces are detected       
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32) #batch prediction to make prediction faster
        
    return (locs, preds) #locations of the faces and their predictions

In [4]:
print("[INFO] Loading face_detector model...")
prototxtPath = r"models\face_detector\deploy.prototxt"
weightsPath = r"models\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

print("[INFO] Loading mask_detector model...")
maskNet = load_model("models\mask_detector\model.h5")

print("[INFO] Starting video stream...")
vs = VideoStream(src=0).start()

print("[INFO] Detecting in progress...")
while True:        
    frame = vs.read()
    frame = imutils.resize(frame, width=frame_dimension)
    
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
        
    for (box, pred) in zip(locs, preds):
        
        (startX, startY, endX, endY) = box #face box
        (mask, withoutMask) = pred #prediction
        
        label = ""
        
        if(mask > withoutMask):
            label = mask_label
        else:
            label = no_mask_label
        
        color = (0,0,0)
        
        if(label == mask_label):
            color = green_color
        else:
            color = red_color
        
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100) # add the probability

        # display the label and bounding box rectangle on the output frame
        cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    
    cv2.imshow("Detecting Mask", frame) # show the output frame
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord(quit_key):
        break

print("[INFO] cleaning up...")
cv2.destroyAllWindows()
vs.stop()
vs.stream.release()
print("[INFO] cleanup terminated")


[INFO] Loading face_detector model...
[INFO] Loading mask_detector model...
[INFO] Starting video stream...
[INFO] Detecting in progress...
[INFO] cleaning up...
[INFO] cleanup terminated
